## Import Necessary Libraries

In [9]:
import pandas as pd
import tensorflow as tf
import os
from tensorflow import keras
from keras_preprocessing.image import load_img, array_to_img, img_to_array, ImageDataGenerator
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [10]:
datagen = ImageDataGenerator()

In [11]:
train_path =  "./data/fruits-360_dataset/fruits-360/Training"
test_path = "./data/fruits-360_dataset/fruits-360/Test"

In [12]:
golden_1_train_path = "./data/fruits-360_dataset/fruits-360/Training/Apple Golden 1"

In [13]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2, 
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (200, 200),
    batch_size = 32,
    color_mode = 'rgb',
    class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size = (200, 200),
    batch_size = 32,
    color_mode = 'rgb',
    class_mode = 'categorical')

Found 5240 images belonging to 6 classes.
Found 1751 images belonging to 6 classes.


In [15]:
train_class_names = list(train_generator.class_indices.keys())
train_class_names

['Apple Braeburn',
 'Apple Crimson Snow',
 'Apple Golden',
 'Apple Granny Smith',
 'Apple Pink Lady',
 'Apple Red Delicious']

In [16]:
test_class_names = list(test_generator.class_indices.keys())
test_class_names

['Apple Braeburn',
 'Apple Crimson Snow',
 'Apple Golden',
 'Apple Granny Smith',
 'Apple Pink Lady',
 'Apple Red Delicious']

## Begin Modeling